# Training Your Own "Dense Passage Retrieval" Model


Haystack contains all the tools needed to train your own Dense Passage Retrieval model.
This tutorial will guide you through the steps required to create a retriever that is specifically tailored to your domain.

In [ ]:
# Install the latest release of Haystack in your own environment
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

In [ ]:
# Here are some imports that we'll need

from haystack.nodes import DensePassageRetriever
from haystack.utils import fetch_archive_from_http
from haystack.document_stores import InMemoryDocumentStore

## Training Data

DPR training performed using Information Retrieval data.
More specifically, you want to feed in pairs of queries and relevant documents.

To train a model, we will need a dataset that has the same format as the original DPR training data.
Each data point in the dataset should have the following dictionary structure.

``` python
    {
        "dataset": str,
        "question": str,
        "answers": list of str
        "positive_ctxs": list of dictionaries of format {'title': str, 'text': str, 'score': int, 'title_score': int, 'passage_id': str}
        "negative_ctxs": list of dictionaries of format {'title': str, 'text': str, 'score': int, 'title_score': int, 'passage_id': str}
        "hard_negative_ctxs": list of dictionaries of format {'title': str, 'text': str, 'score': int, 'title_score': int, 'passage_id': str}
    }
```

`positive_ctxs` are context passages which are relevant to the query.
In some datasets, queries might have more than one positive context
in which case you can set the `num_positives` parameter to be higher than the default 1.
Note that `num_positives` needs to be lower or equal to the minimum number of `positive_ctxs` for queries in your data.
If you have an unequal number of positive contexts per example,
you might want to generate some soft labels by retrieving similar contexts which contain the answer.

DPR is standardly trained using a method known as in-batch negatives.
This means that positive contexts for a given query are treated as negative contexts for the other queries in the batch.
Doing so allows for a high degree of computational efficiency, thus allowing the model to be trained on large amounts of data.

`negative_ctxs` is not actually used in Haystack's DPR training so we recommend you set it to an empty list.
They were used by the original DPR authors in an experiment to compare it against the in-batch negatives method.

`hard_negative_ctxs` are passages that are not relevant to the query.
In the original DPR paper, these are fetched using a retriever to find the most relevant passages to the query.
Passages which contain the answer text are filtered out.



## Download Data

Download the necessary data for dpr training
The greek translation of the original NQ dataset for dpr training can be downloaded by executing the cell below.


In [ ]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1UA7_ouoR8WDeeNReYrZZS_PIXvGiP7Md
!gdown https://drive.google.com/uc?id=1m52D8QXIxxr5914ApldX3M1A547Z_mGD

## Training DPR from Scratch

The default variables that we provide below are chosen to train a DPR model from scratch.
Here, both passage and query embedding models are initialized using BERT base
and the model is trained using Google's Natural Questions dataset (in a format specialised for DPR).

If you are working in a language other than English,
you will want to initialize the passage and query embedding models with a language model that supports your language
and also provide a dataset in your language.

In [ ]:
# Here are the variables to specify our training data, the models that we use to initialize DPR
# and the directory where we'll be saving the model

train_filename = "nq_train_dpr.json"
dev_filename = "nq_dev_dpr.json"

query_model = "nlpaueb/bert-base-greek-uncased-v1"
passage_model = "nlpaueb/bert-base-greek-uncased-v1"

save_dir = "./dpr"

## Initialization

Here we want to initialize our model either with plain language model weights for training from scratch
or else with pretrained DPR weights for finetuning.
We follow the [original DPR parameters](https://github.com/facebookresearch/DPR#best-hyperparameter-settings)
for their max passage length but set max query length to 64 since queries are very rarely longer.

In [ ]:
## Initialize DPR model

retriever = DensePassageRetriever(
    document_store=InMemoryDocumentStore(),
    query_embedding_model=query_model,
    passage_embedding_model=passage_model,
    max_seq_len_query=64,
    max_seq_len_passage=256,
)

## Training

Let's start training and save our trained model!

On a V100 GPU, you can fit up to batch size 16 so we set gradient accumulation steps to 8 in order
to simulate the batch size 128 of the original DPR experiment.

When `embed_title=True`, the document title is prepended to the input text sequence with a `[SEP]` token
between it and document text.

In [ ]:
# Start training our model and save it when it is finished

retriever.train(
    data_dir=".",
    train_filename=train_filename,
    dev_filename=dev_filename,
    test_filename=dev_filename,
    n_epochs=40,
    batch_size=16,
    grad_acc_steps=8,
    save_dir=save_dir,
    evaluate_every=5000,
    embed_title=False,
    num_positives=1,
    num_hard_negatives=1,
    checkpoint_root_dir = "./model_checkpoints",
    checkpoint_every =5000,
    checkpoints_to_keep= 2,
)

## Loading

Loading our newly trained model is simple!

In [ ]:
reloaded_retriever = DensePassageRetriever.load(load_dir=save_dir, document_store=None)



This notebook is based on the [Haystack](https://github.com/deepset-ai/haystack/) notebook for DPR training.